In [1]:
from modules.statistical import *

modules.preprocessing successfull loaded: 100%|███| 6/6 [00:03<00:00,  1.89it/s]


In [2]:
layers = list(range(1,14))
layers

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]

In [3]:
#analyzer_launcher(outputs_name="outputs", analytical_func=compare_MlC_MCA_for_GLO_PER_GAP, layers=layers, approach=approach, aggregation_f=None,logics=None, alphaConfig=[0.1,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.85])

In [4]:
[
    analyzer_launcher_for_descriptor_rank_plot(
        outputs_name="outputs9", 
        analytical_func=Descriptors_comparison, 
        approach=['mixed'],
        layers=layers, 
        aggregation_f=statistics.mean, 
        top=k, 
        alphaConfig=[0.1,0.3,0.4,0.5,0.6,0.7,0.8,0.9],
        with_class=False
    )  for k in [10,15,20,15,25,30]
]

GERMAN
/Users/djiemboutienctheuvictornico/Documents/MyFolders/ACADEMIC/M2_thesis/scripting/outputs9/GERMAN/0.1/qualitative/model_storage
/Users/djiemboutienctheuvictornico/Documents/MyFolders/ACADEMIC/M2_thesis/scripting/outputs9/GERMAN/0.3/qualitative/model_storage
/Users/djiemboutienctheuvictornico/Documents/MyFolders/ACADEMIC/M2_thesis/scripting/outputs9/GERMAN/0.4/qualitative/model_storage
/Users/djiemboutienctheuvictornico/Documents/MyFolders/ACADEMIC/M2_thesis/scripting/outputs9/GERMAN/0.5/qualitative/model_storage
/Users/djiemboutienctheuvictornico/Documents/MyFolders/ACADEMIC/M2_thesis/scripting/outputs9/GERMAN/0.6/qualitative/model_storage
/Users/djiemboutienctheuvictornico/Documents/MyFolders/ACADEMIC/M2_thesis/scripting/outputs9/GERMAN/0.7/qualitative/model_storage
/Users/djiemboutienctheuvictornico/Documents/MyFolders/ACADEMIC/M2_thesis/scripting/outputs9/GERMAN/0.8/qualitative/model_storage
/Users/djiemboutienctheuvictornico/Documents/MyFolders/ACADEMIC/M2_thesis/scripting

/Users/djiemboutienctheuvictornico/Documents/MyFolders/ACADEMIC/M2_thesis/scripting/.env/lib/python3.9/site-packages/pandas/plotting/_matplotlib/core.py:513: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = self.plt.figure(figsize=self.figsize)


Directory '/Users/djiemboutienctheuvictornico/Documents/MyFolders/ACADEMIC/M2_thesis/scripting/analyze_outputs9_made_on_2024_05_20_15H/descriptor/shap/15' already exists.


/Users/djiemboutienctheuvictornico/Documents/MyFolders/ACADEMIC/M2_thesis/scripting/modules/statistical.py:1482: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  pl.figure(figsize=(width, height))


Directory '/Users/djiemboutienctheuvictornico/Documents/MyFolders/ACADEMIC/M2_thesis/scripting/analyze_outputs9_made_on_2024_05_20_15H/descriptor/shap/15' already exists.
Directory '/Users/djiemboutienctheuvictornico/Documents/MyFolders/ACADEMIC/M2_thesis/scripting/analyze_outputs9_made_on_2024_05_20_15H/descriptor/shap/15' already exists.
Directory '/Users/djiemboutienctheuvictornico/Documents/MyFolders/ACADEMIC/M2_thesis/scripting/analyze_outputs9_made_on_2024_05_20_15H/descriptor/shap/15' already exists.
GERMAN
/Users/djiemboutienctheuvictornico/Documents/MyFolders/ACADEMIC/M2_thesis/scripting/outputs9/GERMAN/0.1/qualitative/model_storage
/Users/djiemboutienctheuvictornico/Documents/MyFolders/ACADEMIC/M2_thesis/scripting/outputs9/GERMAN/0.3/qualitative/model_storage
/Users/djiemboutienctheuvictornico/Documents/MyFolders/ACADEMIC/M2_thesis/scripting/outputs9/GERMAN/0.4/qualitative/model_storage
/Users/djiemboutienctheuvictornico/Documents/MyFolders/ACADEMIC/M2_thesis/scripting/output

...

<Figure size 1500x100 with 0 Axes>

<Figure size 1500x100 with 0 Axes>

<Figure size 1500x100 with 0 Axes>

<Figure size 1500x100 with 0 Axes>

<Figure size 1500x100 with 0 Axes>